In [318]:
# import mysql.connector
import pymysql
from pymysql.cursors import DictCursor
conn_1=pymysql.connect(host="localhost",
                            database="digital_factory_v1",
                            user="root",
                            password="")
conn_2=pymysql.connect(host="localhost",
                            database="digital_factory_v1_completed",
                            user="root",
                            password="")
cur_1=conn_1.cursor(DictCursor)
cur_2=conn_2.cursor()


In [319]:
cur_1.execute(f"select * from master_shifts")
val=cur_1.fetchall()

In [320]:
val[0]

{'shift_id': 1,
 'company_id': 1,
 'no_of_shifts': 3,
 'shift1_start_time': datetime.datetime(1900, 1, 1, 7, 0),
 'shift2_start_time': datetime.datetime(1900, 1, 1, 15, 30),
 'shift3_start_time': datetime.datetime(1900, 1, 1, 23, 30),
 'mill_date': datetime.datetime(2022, 8, 6, 0, 0),
 'mill_shift': '1',
 'created_on': datetime.datetime(2019, 3, 28, 21, 35, 14),
 'created_by': 0,
 'modified_on': '2020-08-04 18:41:32',
 'modified_by': 1}

In [321]:
from datetime import timedelta

mill_shift=int(val[0]["mill_shift"])-1
mill_date=val[0]["mill_date"]
# print(mill_shift)
if mill_shift==0:
    mill_shift=val[0]["no_of_shifts"]
    mill_date-=timedelta(2)

print(mill_date)
print(mill_shift)




2022-08-04 00:00:00
3


In [322]:
mill_month={1:"01",2:"02",3:"03",4:"04",5:"05",6:"06",7:"07",8:"08",9:"09",10:"10",11:"11",12:"12"}

table_name="production_"+mill_month[mill_date.month]+str(mill_date.year)+"_"+"shift"
mill_date=str(mill_date).split()[0]

print(str(mill_date))

2022-08-04


In [323]:
print(f"select exists (select table_name from information_schema.tables where table_schema='digital_factory_v1_completed' and table_name='{table_name}')")

select exists (select table_name from information_schema.tables where table_schema='digital_factory_v1_completed' and table_name='production_082022_shift')


In [324]:
cur_2.execute(f"select exists (select table_name from information_schema.tables where table_schema='digital_factory_v1_completed' and table_name='{table_name}')")
state=cur_2.fetchall()
print(state[0][0])

1


In [325]:
print(table_name)
cur_2.execute(f"select exists (select table_name from information_schema.tables where table_schema='digital_factory_v1_completed' and table_name='{table_name}')")
state = cur_2.fetchall()
print(state[0][0])
print("hii")


cur_2.close()
cur_2 = conn_2.cursor()

if state[0][0] == 0:
	cur_2.execute(f"""create table {table_name} SELECT cp.department_id,cp.machine_id,cp.mill_date,cp.mill_shift,
					CASE WHEN SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2) >0 THEN 
						ROUND((SUM(cp.run_time)/SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2))*100,1) 
					ELSE 0 END AR,
					
					CASE WHEN (mp.cycle_time_spec*(SUM(cp.run_time)/60)) >0 THEN 
						CASE WHEN ROUND((((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw)))*100,1) > 100 THEN 100 ELSE ROUND((((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw)))*100,1) END 
					ELSE 0 END AS PR,
					
					CASE WHEN SUM(cp.actual) >0 THEN
						ROUND((SUM(((cp.actual*mp.part_per_stoke)/mp.division_factor)-cp.rejection)/SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))*100,1) 
					ELSE 0 END AS QR,
					
					ROUND(((CASE WHEN SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2) >0 THEN 
						(SUM(cp.run_time)/SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2))
					ELSE 0 END)*
					(CASE WHEN (mp.cycle_time_spec*(SUM(cp.run_time)/60)) >0 THEN 
						CASE WHEN (((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw))) > 1 THEN 1 ELSE (((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw))) END
					ELSE 0 END)*
					(CASE WHEN SUM(cp.actual) >0 THEN
						(SUM(((cp.actual*mp.part_per_stoke)/mp.division_factor)-cp.rejection)/SUM((cp.actual*mp.part_per_stoke)/mp.division_factor)) 
					ELSE 0 END))*100,1) AS OEE,
					
					SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw) AS pr_by_val,
					
					ROUND(SUM(cp.target*mp.draw),0) AS target,
					ROUND(SUM((cp.actual*mp.part_per_stoke)/mp.division_factor),0) AS actual,
					ROUND(SUM((cp.actual*mp.part_per_stoke)/mp.division_factor)*mp.width,0) AS sq_mtr,
					
					ROUND(SUM(((cp.actual*mp.part_per_stoke)/mp.division_factor)-(cp.rejection)),0) AS ok_qty,
					SUM(cp.run_time) AS running_time,
					SUM(cp.loss_time_1+cp.loss_time_2) AS loss_time
					
					FROM digital_factory_v1_completed.production_082022 cp,
					digital_factory_v1.master_product mp
			
					WHERE cp.product_id=mp.product_id
					 AND cp.mill_shift={mill_shift} and cp.mill_date="{mill_date}"
					GROUP BY machine_id""")
	conn_2.commit()
	result=cur_2.fetchall()
	print(result)
	cur_2.execute(f"alter table {table_name} add column id int primary key auto_increment")

elif state[0][0]==1:
	print("hello")
	cur_2.execute(f"""INSERT INTO {table_name}(department_id,machine_id,mill_date,mill_shift,AR,PR,QR,OEE,pr_by_val,target,actual,sq_mtr,ok_qty,running_time,loss_time)
					SELECT cp.department_id,cp.machine_id,cp.mill_date,cp.mill_shift,
					CASE WHEN SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2) >0 THEN 
						ROUND((SUM(cp.run_time)/SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2))*100,1) 
					ELSE 0 END AR,
					
					CASE WHEN (mp.cycle_time_spec*(SUM(cp.run_time)/60)) >0 THEN 
						CASE WHEN ROUND((((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw)))*100,1) > 100 THEN 100 ELSE ROUND((((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw)))*100,1) END 
					ELSE 0 END AS PR,
					
					CASE WHEN SUM(cp.actual) >0 THEN
						ROUND((SUM(((cp.actual*mp.part_per_stoke)/mp.division_factor)-cp.rejection)/SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))*100,1) 
					ELSE 0 END AS QR,
					
					ROUND(((CASE WHEN SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2) >0 THEN 
						(SUM(cp.run_time)/SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2))
					ELSE 0 END)*
					(CASE WHEN (mp.cycle_time_spec*(SUM(cp.run_time)/60)) >0 THEN 
						CASE WHEN (((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw))) > 1 THEN 1 ELSE (((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw))) END
					ELSE 0 END)*
					(CASE WHEN SUM(cp.actual) >0 THEN
						(SUM(((cp.actual*mp.part_per_stoke)/mp.division_factor)-cp.rejection)/SUM((cp.actual*mp.part_per_stoke)/mp.division_factor)) 
					ELSE 0 END))*100,1) AS OEE,
					
					SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw) AS pr_by_val,
					
					ROUND(SUM(cp.target*mp.draw),0) AS target,
					ROUND(SUM((cp.actual*mp.part_per_stoke)/mp.division_factor),0) AS actual,
					ROUND(SUM((cp.actual*mp.part_per_stoke)/mp.division_factor)*mp.width,0) AS sq_mtr,
					
					ROUND(SUM(((cp.actual*mp.part_per_stoke)/mp.division_factor)-(cp.rejection)),0) AS ok_qty,
					SUM(cp.run_time) AS running_time,
					SUM(cp.loss_time_1+cp.loss_time_2) AS loss_time
					
					FROM digital_factory_v1_completed.production_082022 cp,
					digital_factory_v1.master_product mp
			
					WHERE cp.product_id=mp.product_id
					AND cp.mill_date="{mill_date}" AND cp.mill_shift={mill_shift}
					GROUP BY machine_id""")
	conn_2.commit()
	result=cur_2.fetchall()
	print(result)
		
			



production_082022_shift
1
hii
hello
()


In [291]:
print(f"""INSERT INTO {table_name}(department_id,machine_id,mill_date,mill_shift,AR,PR,QR,OEE,pr_by_val,target,actual,sq_mtr,ok_qty,running_time,loss_time)
					SELECT cp.department_id,cp.machine_id,cp.mill_date,cp.mill_shift,
					CASE WHEN SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2) >0 THEN 
						ROUND((SUM(cp.run_time)/SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2))*100,1) 
					ELSE 0 END AR,
					
					CASE WHEN (mp.cycle_time_spec*(SUM(cp.run_time)/60)) >0 THEN 
						CASE WHEN ROUND((((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw)))*100,1) > 100 THEN 100 ELSE ROUND((((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw)))*100,1) END 
					ELSE 0 END AS PR,
					
					CASE WHEN SUM(cp.actual) >0 THEN
						ROUND((SUM(((cp.actual*mp.part_per_stoke)/mp.division_factor)-cp.rejection)/SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))*100,1) 
					ELSE 0 END AS QR,
					
					ROUND(((CASE WHEN SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2) >0 THEN 
						(SUM(cp.run_time)/SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2))
					ELSE 0 END)*
					(CASE WHEN (mp.cycle_time_spec*(SUM(cp.run_time)/60)) >0 THEN 
						CASE WHEN (((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw))) > 1 THEN 1 ELSE (((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw))) END
					ELSE 0 END)*
					(CASE WHEN SUM(cp.actual) >0 THEN
						(SUM(((cp.actual*mp.part_per_stoke)/mp.division_factor)-cp.rejection)/SUM((cp.actual*mp.part_per_stoke)/mp.division_factor)) 
					ELSE 0 END))*100,1) AS OEE,
					
					SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw) AS pr_by_val,
					
					ROUND(SUM(cp.target*mp.draw),0) AS target,
					ROUND(SUM((cp.actual*mp.part_per_stoke)/mp.division_factor),0) AS actual,
					ROUND(SUM((cp.actual*mp.part_per_stoke)/mp.division_factor)*mp.width,0) AS sq_mtr,
					
					ROUND(SUM(((cp.actual*mp.part_per_stoke)/mp.division_factor)-(cp.rejection)),0) AS ok_qty,
					SUM(cp.run_time) AS running_time,
					SUM(cp.loss_time_1+cp.loss_time_2) AS loss_time
					
					FROM digital_factory_v1_completed.production_082022 cp,
					digital_factory_v1.master_product mp
			
					WHERE cp.product_id=mp.product_id
					AND cp.mill_date="{mill_date}" AND cp.mill_shift={mill_shift}
					GROUP BY machine_id""")

INSERT INTO production_082022_shift(department_id,machine_id,mill_date,mill_shift,AR,PR,QR,OEE,pr_by_val,target,actual,sq_mtr,ok_qty,running_time,loss_time)
					SELECT cp.department_id,cp.machine_id,cp.mill_date,cp.mill_shift,
					CASE WHEN SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2) >0 THEN 
						ROUND((SUM(cp.run_time)/SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2))*100,1) 
					ELSE 0 END AR,
					
					CASE WHEN (mp.cycle_time_spec*(SUM(cp.run_time)/60)) >0 THEN 
						CASE WHEN ROUND((((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw)))*100,1) > 100 THEN 100 ELSE ROUND((((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw)))*100,1) END 
					ELSE 0 END AS PR,
					
					CASE WHEN SUM(cp.actual) >0 THEN
						ROUND((SUM(((cp.actual*mp.part_per_stoke)/mp.division_factor)-cp.rejection)/SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))*100,1) 
					ELSE 0 E

In [309]:
cur_2.execute("drop table production_082022_shift")

0

In [ ]:
# cur.execute("select * from parameters")
# par_form=cur.fetchall()
# print(par_form)
# if mill_shift==1 and mill_date.day==1:
#     table_name="production_"+str(mill_date.month)+str(mill_date.year)+"_"
#     print(table_name)
#     cur.execute("select * from parameters")
#     par_form=cur.fetchall()
    
#     base_str=""
#     for val in range(0,len(par_form)):
#         if val==0:
#             base_str+=f"{par_form[val][2]} as {par_form[val][1]} "
#             print(base_str)
#         else:
#             base_str+=f",{par_form[val][2]} as {par_form[val][1]} "
#             print(base_str)

#     print(base_str)
#     print(f"create table {table_name}shift select {base_str} from production_052022")
    
#     cur.execute(f"create table {table_name}shift select {base_str} from production_052022")

# else:
#     cur.execute("select column_name from information_schema.columns where table_schema='si_df_completed' and table_name='production_52022_shift'")
#     column_1=cur.fetchall()
#     cur.execute("select parameter from parameters")
#     column_2=cur.fetchall()
#     print(column_1)
#     print(column_2)
#     print(column_2==column_1)
#     result=set(column_2)-set(column_1)
#     print(list(result))
#     # print(set(column_2)-set(column_1))
    
    

# # cur.execute("select * from parameters")
# # val=cur.fetchall()
# # base_str="select "
# # print(len(val))

# # for i in val:
# #     temp_str=f"{i[2]} as {i[1]}"
# #     base_str+=temp_str

# # print("select "+ base_str+"from production_052022")

